In [ ]:
import instaloader
import pandas as pd
import numpy as np
from skimage import io
from requests.exceptions import RequestException
import cv2

In [4]:
def load_data(usernames, filepath):

    posts_data = []
    max_posts = 5
    
    for username in usernames:
        post_count = 0
        profile = instaloader.Profile.from_username(L.context, username)
        for post in profile.get_posts():
            if post_count >= max_posts:
                break
            post_data = {
                'name': username,
                'date': post.date_utc,
                'url': post.url,
                'caption': post.caption,
                'likes': post.likes
            }
            posts_data.append(post_data)
            post_count += 1

    df = pd.DataFrame(posts_data)
    df.to_csv(filepath, index=False)

L = instaloader.Instaloader()

In [ ]:
usernames = ['brumarquezine', 'luissuarez9', 'hudabeauty']
load_data(usernames, 'names1.csv')

usernames = ['ayutingting92', 'marinaruybarbosa', 'lelepons']
load_data(usernames, 'names2.csv')

usernames =['marcelotwelve', 'harrystyles', 'danbilzerian']
load_data(usernames, 'names3.csv')

# and about 50 more users...


In [ ]:
names1 = pd.read_csv('names1.csv')
names2 = pd.read_csv('names2.csv')
names3 = pd.read_csv('names3.csv')
names4 = pd.read_csv('names4.csv')
#...
res_df = names1
res_df = pd.concat([res_df, names2], ignore_index=True)
res_df = pd.concat([res_df, names3], ignore_index=True)
res_df = pd.concat([res_df, names4], ignore_index=True)
#...


In [ ]:
call_count = 0
def load_image(url, retries=3):
    for attempt in range(retries):
        try:
            image = io.imread(url)
            global call_count
            call_count += 1
            print("Загрузка прошла успешно", call_count)
            return cv2.resize(image, (224, 224))
        except RequestException as e:
            print(f"Ошибка загрузки изображения {url}")
            if attempt == retries - 1:
                raise e 

res_df['images'] = res_df['url'].apply(lambda x: load_image(x))
images_array = np.array(res_df['images'].tolist())
np.save('images_array.npy', images_array)
res_df.to_csv('res_df.csv', index=False)